# Creating the MODFLOW 6 model
This notebook generates the input data for the groundwater model in the coupling with MetaSWAP.

In [ ]:
import flopy
import os
import numpy as np

model_dir = os.path.join(os.getcwd(),"mf6_data")
model_name = 'hooghoudt'

Set the model parameters:

In [ ]:
# tdis
nper = 365
tdis_rc = []
for i in range(nper):
    tdis_rc.append((1., 1, 1))

# solver data
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-9, 1e-3, 0.97

# model spatial discretization
nlay, nrow, ncol = 3, 9, 9

# cell spacing
delr = 10.0
delc = 10.0
area = delr * delc

# top/bot of the aquifer
tops = [0.0, -5.0, -25.0, -325.0]

# hydraulic conductivity
hk = [10., 0.001, 1.0]

# boundary heads
h_left = -1.5
h_right = -1.5
ghb_cond = 0.5

# initial head
h_start = -1.2

Now generate the boundary data, we use GHB to artificially place the edge of the domain further away and have an increased effect on the bulging behavior of the groundwater table:

In [ ]:
left_ghb = [[(ilay,irow,0), h_left, ghb_cond] for ilay in range(nlay) for irow in range(nrow)]
right_ghb = [[(ilay,irow,ncol-1), h_right, ghb_cond] for ilay in range(nlay) for irow in range(nrow)]
ghb_data = left_ghb + right_ghb
ghb_spd = {0: ghb_data}

The following data are placeholders into which the coupled MetaSWAP variables will be set. Their actual value is not relevant here.

In [ ]:
# placeholder recharge data for MetaSWAP coupling,
# no coupling to MSW on the boundary
rch_spd = {}
rch_arr = [[(0,irow,icol), 0.0] for irow in range(nrow) for icol in range(1,ncol-1)]
rch_spd[0] = rch_arr

# single well for irrigation extraction
wel_spd = {}
wel_data = [[(2, 2, 4), 0.0]]
wel_spd[0] = wel_data

Now we can use the Flopy toolchain to generate the model:

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name="mfsim", version='mf6',
                                 exe_name='mf6',
                                 sim_ws=model_dir, memory_print_option='all')
# create tdis package

tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                             nper=nper, perioddata=tdis_rc)

# create iterative model solution and register the gwf model with it
ims = flopy.mf6.ModflowIms(sim,
                           print_option='SUMMARY',
                           outer_hclose=hclose,
                           outer_maximum=nouter,
                           under_relaxation='DBD',
                           inner_maximum=ninner,
                           inner_hclose=hclose, rcloserecord=rclose,
                           linear_acceleration='BICGSTAB',
                           relaxation_factor=relax)

# create gwf model
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, save_flows=True)

dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              delr=delr, delc=delc,
                              top=tops[0], botm=tops[1:])

# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, strt=h_start)

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, save_flows=True,
                              icelltype=0,
                              k=hk)

# ghb file
ghb = flopy.mf6.ModflowGwfghb(gwf, stress_period_data=ghb_spd)

# recharge file
rch = flopy.mf6.ModflowGwfrch(gwf, stress_period_data=rch_spd, pname='RCH_MSW')

# well for sprinkling to MSW
wel = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, pname='WELLS_MSW')

# storage, iconvert=0 means confined
sto = flopy.mf6.ModflowGwfsto(gwf, iconvert=0, ss=[0., 1.e-5, 1.e-5], transient={0: True})

# output control
oc = flopy.mf6.ModflowGwfoc(gwf,
                            head_filerecord='{}.hds'.format(model_name),
                            headprintrecord=[
                                ('COLUMNS', 10, 'WIDTH', 15,
                                 'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL')],
                            printrecord=[('BUDGET', 'ALL')])

sim.write_simulation()
